In [1]:
import pandas as pd
import seaborn as sns

In [7]:
data = pd.read_csv('../OUTPUT/numerical_features.csv')
data.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.21,0,2,4,63.0,57.0,6.73,6.70,4.23,6134
1,0.28,2,6,2,64.0,56.0,4.14,4.17,2.66,532
2,0.42,1,4,3,61.2,58.0,4.86,4.82,2.96,1103
3,0.26,0,2,0,61.1,57.0,4.16,4.12,2.53,600
4,1.10,3,3,5,63.4,57.0,6.52,6.55,4.14,4997
